# Shadow Rollout with Seldon and Ambassador

This notebook shows how you can deploy "shadow" deployments to direct traffic not only to the main Seldon Deployment but also to a shadow deployment whose response will be dicarded. This allows you to test new models in a production setting and with production traffic and anlalyse how they perform before putting them live.

These are useful when you want to test a new model or higher latency inference piepline (e.g., with explanation components) with production traffic but without affecting the live deployment.


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

In [54]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [55]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


In [56]:
from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

In [57]:
VERSION = !cat ../../../version.txt
VERSION = VERSION[0]
VERSION

'1.19.0-dev'

## Launch main model

We will create a very simple Seldon Deployment with a dummy model image `seldonio/mock_classifier:1.0`. This deployment is named `example`.

In [58]:
%%writetemplate ambassador-example-model.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
  name: ambassador-example
spec:
  name: production-model
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:{VERSION}
          imagePullPolicy: IfNotPresent
          name: classifier
        terminationGracePeriodSeconds: 1
    graph:
      children: []
      endpoint:
        type: REST
      name: classifier
      type: MODEL
    name: default
    replicas: 1


In [59]:
!kubectl apply -f ambassador-example-model.yaml

seldondeployment.machinelearning.seldon.io/ambassador-example created


In [60]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=ambassador-example -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "ambassador-example-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "ambassador-example-default-0-classifier" successfully rolled out


### Get predictions

In [61]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(deployment_name="ambassador-example", namespace="seldon")

#### REST Request

In [62]:
r = sc.predict(gateway="ambassador", transport="rest")
print(r)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.75223221557562092
  }
}

Response:
{'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.10299006125367247]}}, 'meta': {'requestPath': {'classifier': 'seldonio/mock_classifier:1.19.0-dev'}}}


## Launch Shadow

We will now create a new Seldon Deployment for our Shadow deployment with a new model.

In [63]:
%%writetemplate ambassador-example-model.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
  name: ambassador-example
spec:
  name: shadow-model
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:{VERSION}
          imagePullPolicy: IfNotPresent
          name: classifier
        terminationGracePeriodSeconds: 1
    graph:
      children: []
      endpoint:
        type: REST
      name: classifier
      type: MODEL
    name: default
    replicas: 1
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:{VERSION}
          imagePullPolicy: IfNotPresent
          name: classifier
    graph:
      children: []
      endpoint:
        type: REST
      name: classifier
      type: MODEL
    name: shadow
    replicas: 1
    shadow: true
    traffic: 100


In [64]:
!kubectl apply -f ambassador-example-model.yaml

seldondeployment.machinelearning.seldon.io/ambassador-example configured


In [65]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=ambassador-example -o jsonpath='{.items[0].metadata.name}')
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=ambassador-example -o jsonpath='{.items[1].metadata.name}')

deployment "ambassador-example-default-0-classifier" successfully rolled out
deployment "ambassador-example-shadow-0-classifier" successfully rolled out


Let's send a bunch of requests to the endpoint.

In [66]:
for i in range(10):
    r = sc.predict(gateway="ambassador", transport="rest")

In [67]:
default_count = !kubectl logs $(kubectl get pod -lseldon-app=ambassador-example-default -o jsonpath='{.items[0].metadata.name}') classifier | grep "root.predict" | wc -l

In [68]:
shadow_count = !kubectl logs $(kubectl get pod -lseldon-app=ambassador-example-shadow -o jsonpath='{.items[0].metadata.name}') classifier | grep "root.predict" | wc -l

In [69]:
print(shadow_count)
print(default_count)
assert int(shadow_count[0]) == 10
assert int(default_count[0]) == 11

['10']
['11']


## TearDown

In [70]:
!kubectl delete -f ambassador-example-model.yaml

seldondeployment.machinelearning.seldon.io "ambassador-example" deleted
